<a href="https://colab.research.google.com/github/tb-harris/ASCtest0/blob/main/09_Figuring_Out_the_Feature_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the feature extractor library:

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install neuron_morphology

In [ ]:
import sys
sys.path.insert(0, "../")

from io import StringIO
import copy
import matplotlib.pyplot as plt

import neuron_morphology.swc_io as swcio
from neuron_morphology.morphology import Morphology
from neuron_morphology.swc_io import morphology_from_swc
from neuron_morphology.feature_extractor.data import Data
from neuron_morphology.feature_extractor.feature_extractor import FeatureExtractor
from neuron_morphology.features.default_features import default_features
from neuron_morphology.constants import (
    SOMA, AXON, BASAL_DENDRITE, APICAL_DENDRITE
)

import json
import numpy as np
import neuron_morphology.feature_extractor.feature_writer as fw

2024-07-15 13:36:58,563 numexpr.utils INFO     NumExpr defaulting to 2 threads.


Load in cell metadata and manifest file containing reconstruction file names

In [ ]:
import pandas as pd

#metadata
path = '/content/drive/Shareddrives/Lisman Laboratory/Data 2023/Allen Brain Institute Data/20200711_patchseq_metadata_mouse.csv'
metadata = pd.read_csv(path)
# manifest
path1 = '/content/drive/Shareddrives/Lisman Laboratory/Data 2023/Allen Brain Institute Data/2021-09-13_mouse_file_manifest.csv'
manifest = pd.read_csv(path1)

Get the URL to download each of our 573 neuron reconstruction files:

In [ ]:
#get 573 morph file and their ids
swc_urls = manifest.loc[
    (manifest["file_type"] == "transformed_swc")
]

archive_url = swc_urls["archive_uri"].values

Download the reconstruction files:

In [ ]:
#5m 10s
for url in archive_url:
  !wget {url}

Streaming output truncated to the last 5000 lines.

718636190_transform 100%[===================>] 402.48K  --.-KB/s    in 0.1s    

2024-07-15 13:38:59 (3.41 MB/s) - ‘718636190_transformed.swc’ saved [412135]

--2024-07-15 13:38:59--  ftp://download.brainlib.org:8811/biccn/zeng/pseq/morph/200526/718640845_transformed.swc
           => ‘718640845_transformed.swc’
Resolving download.brainlib.org (download.brainlib.org)... 192.231.243.62
Connecting to download.brainlib.org (download.brainlib.org)|192.231.243.62|:8811... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /bil/data/biccn/zeng/pseq/morph/200526 ... done.
==> SIZE 718640845_transformed.swc ... 1790225
==> PASV ... done.    ==> RETR 718640845_transformed.swc ... done.
Length: 1790225 (1.7M) (unauthoritative)

718640845_transform 100%[===================>]   1.71M  1.89MB/s    in 0.9s    

2024-07-15 13:39:00 (1.89 MB/s) - ‘718640845_transformed.swc’ save

In [ ]:
#get morph files
swc_path = swc_urls["file_name"].values

import os
# convert relative file paths to absolute file paths
swc_paths = [os.path.abspath(path) for path in swc_urls["file_name"].values.tolist()]

morphologies = []
# pass the file paths to morphology_from_swc() function
for path in swc_paths:
  morph = morphology_from_swc(path)
  morphologies.append(morph)

### Step 1: Import our features

Find our relevant feature(s) from the [documentation](https://neuron-morphology.readthedocs.io/en/latest/autoapi/neuron_morphology/features/index.html) and import

In [9]:
from neuron_morphology.features.path import max_path_distance
from neuron_morphology.features.branching.bifurcations import mean_bifurcation_angle_local

### Step 2 register our features
Register the features we want to use:

In [10]:
from neuron_morphology.feature_extractor.marked_feature import specialize
from neuron_morphology.feature_extractor.feature_specialization import NEURITE_SPECIALIZATIONS

fe = FeatureExtractor()
fe.register_features([
    specialize(max_path_distance, NEURITE_SPECIALIZATIONS),
    specialize(mean_bifurcation_angle_local, NEURITE_SPECIALIZATIONS)
])

### Step 3: Extract features

In [ ]:
from neuron_morphology.feature_extractor.utilities import unnest

# Extract the features from a single neuron morphology object
def extract_features(neuron_morphology):
  data = Data(neuron_morphology)

  feature_extraction_run = fe.extract(data)
  results = feature_extraction_run.results

  unnest(results)
  return results

Create a data frame by running the *extract_features()* function on each neuron morphology.

In [ ]:
features = pd.DataFrame(
    (extract_features(neuron) for neuron in morphologies),
    index=swc_urls["cell_specimen_id"].astype(int)
)

2024-07-15 13:43:11,122 root         INFO     skipping mark (validation failed): type
2024-07-15 13:43:11,131 root         INFO     selected marks: <bound method FeatureExtractionRun.select_marks of <neuron_morphology.feature_extractor.feature_extraction_run.FeatureExtractionRun object at 0x7fa8b149f580>>
2024-07-15 13:43:11,132 root         INFO     skipping feature: apical_dendrite.max_path_distance. Found extra marks: ['RequiresApical']
2024-07-15 13:43:11,136 root         INFO     skipping feature: apical_dendrite.mean_bifurcation_angle_local. Found extra marks: ['RequiresApical']
2024-07-15 13:43:11,138 root         INFO     selected features: ['basal_dendrite.max_path_distance', 'axon.max_path_distance', 'all_neurites.max_path_distance', 'dendrite.max_path_distance', 'basal_dendrite.mean_bifurcation_angle_local', 'axon.mean_bifurcation_angle_local', 'all_neurites.mean_bifurcation_angle_local', 'dendrite.mean_bifurcation_angle_local']
2024-07-15 13:43:11,414 root         INFO     

In [ ]:
features